## Sudoku Hinweise

#### Einlesen der Daten

In [2]:
import numpy as np
f = open('sudoku0.txt',"r",encoding='utf-8')
data = f.read()                              # die ganze Eingabe in einen String lesen

all =  [int(x) for x in data.split()]        # der Zeilenvorschub verschwindet bei der Umwandlung in int
a = np.array(all[:81]).reshape(9,9)          # erstes Sudoku
b = np.array(all[81:]).reshape(9,9)          # zweites Sudoku

a

array([[6, 0, 0, 0, 8, 0, 0, 0, 1],
       [0, 9, 0, 4, 0, 0, 6, 0, 0],
       [0, 0, 0, 0, 9, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 6, 0, 0, 0, 0, 5],
       [3, 2, 7, 5, 0, 0, 0, 0, 8],
       [0, 0, 0, 0, 7, 0, 0, 0, 0],
       [0, 0, 6, 8, 0, 3, 9, 7, 0],
       [0, 0, 0, 0, 0, 0, 0, 8, 0]])

#### Varianten erzeugen

Wenn wir Teile einer Matrix bearbeiten, machen wir zunächst eine Kopie der Ausgangsmatrix, um Seiteneffekte zu vermeiden.

In [3]:
# Vertauschen zweier Zeilen
a1 = a.copy()                
a1[[0,2],:] = a1[[2,0],:]
print(a1)

[[0 0 0 0 9 0 0 0 2]
 [0 9 0 4 0 0 6 0 0]
 [6 0 0 0 8 0 0 0 1]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 6 0 0 0 0 5]
 [3 2 7 5 0 0 0 0 8]
 [0 0 0 0 7 0 0 0 0]
 [0 0 6 8 0 3 9 7 0]
 [0 0 0 0 0 0 0 8 0]]


In [4]:
# Vertauschen des oberen und unteren 3er-Zeilenblocks
a1 = a.copy()                
a1[[0,1,2,6,7,8],:] = a1[[6,7,8,0,1,2],:]
print(a1)

[[0 0 0 0 7 0 0 0 0]
 [0 0 6 8 0 3 9 7 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 6 0 0 0 0 5]
 [3 2 7 5 0 0 0 0 8]
 [6 0 0 0 8 0 0 0 1]
 [0 9 0 4 0 0 6 0 0]
 [0 0 0 0 9 0 0 0 2]]


In [5]:
# Vertauschen zweier Spalten
a1 = a.copy()                
a1[:,[0,1]] = a1[:,[1,0]]
print(a1)

[[0 6 0 0 8 0 0 0 1]
 [9 0 0 4 0 0 6 0 0]
 [0 0 0 0 9 0 0 0 2]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 6 0 0 0 0 5]
 [2 3 7 5 0 0 0 0 8]
 [0 0 0 0 7 0 0 0 0]
 [0 0 6 8 0 3 9 7 0]
 [0 0 0 0 0 0 0 8 0]]


In [6]:
# Vertauschen des linken und rechten 3er-Spaltenblocks
a1 = a.copy()                
a1[:,[0,1,2,6,7,8]] = a1[:,[6,7,8,0,1,2]]
print(a1)

[[0 0 1 0 8 0 6 0 0]
 [6 0 0 4 0 0 0 9 0]
 [0 0 2 0 9 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 5 6 0 0 0 0 0]
 [0 0 8 5 0 0 3 2 7]
 [0 0 0 0 7 0 0 0 0]
 [9 7 0 8 0 3 0 0 6]
 [0 8 0 0 0 0 0 0 0]]


In [7]:
# Drehen um 90 Grad im Uhrzeigersinn
a1 = a.copy()
a1 = a1[::-1,:].T
print(a1)

[[0 0 0 3 0 0 0 0 6]
 [0 0 0 2 0 0 0 9 0]
 [0 6 0 7 0 0 0 0 0]
 [0 8 0 5 6 0 0 4 0]
 [0 0 7 0 0 0 9 0 8]
 [0 3 0 0 0 1 0 0 0]
 [0 9 0 0 0 0 0 6 0]
 [8 7 0 0 0 0 0 0 0]
 [0 0 0 8 5 0 2 0 1]]


#### Anzahl Varianten

- Für die Permutation der Spaltenblöcke gibt es 6 Möglichkeiten: $*6$  
- Pro Spaltenblock gibt es für die Permutation der Spalten 6 Möglichkeiten: $6*6*6$  
- Für die Permutation der Zeilenblöcke gibt es 6 Möglichkeiten: $*6$
- Pro Zeilenblock gibt es für die Permutation der Zeilen 6 Möglichkeiten $6*6*6$ 
- Eine 90 Grad-Drehung kann stattfinden oder nicht: $*2$



In [8]:
# Anzahl Möglicher Varianten (ohne Permutation der Ziffern)
(6**8)*2

3359232

In [9]:
# Anzahl Möglicher Varianten (mit Permutation der Ziffern)
import math
(6**8)*2 * math.factorial(9)

1218998108160

#### Mögliches Vorgehen

Wir prüfen zunächst ohne Vertauschung von Ziffern, ob es eine Variante gibt, die die Leerstellen an derselben Stelle hat wie das zweite Sudoku. Dazu müssen wir 3359232 Varianten überprüfen. Das könnte vielleicht noch in vertretbarer Zeit möglich sein.

Erst wenn wir solche Varianten gefunden haben, prüfen wir die Permutation der Ziffern.

Wenn das Verfahren zu lange dauert, können wir einen backtracking-Algorithmus probieren. Wir können schrittweise überprüfen, ob wir auf dem richtigen Weg sind, indem wir die Nullstellen für die Bereiche zählen, die wir gerade bearbeiten.

#### Nützliches mit numpy

In [26]:
a = np.array([[6, 0, 0, 0, 8, 0, 0, 0, 1],
       [0, 9, 0, 4, 0, 0, 6, 0, 0],
       [0, 0, 0, 0, 9, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 6, 0, 0, 0, 0, 5],
       [3, 2, 7, 5, 0, 0, 0, 0, 8],
       [0, 0, 0, 0, 7, 0, 0, 0, 0],
       [0, 0, 6, 8, 0, 3, 9, 7, 0],
       [0, 0, 0, 0, 0, 0, 0, 8, 0]])
print(a)

[[6 0 0 0 8 0 0 0 1]
 [0 9 0 4 0 0 6 0 0]
 [0 0 0 0 9 0 0 0 2]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 6 0 0 0 0 5]
 [3 2 7 5 0 0 0 0 8]
 [0 0 0 0 7 0 0 0 0]
 [0 0 6 8 0 3 9 7 0]
 [0 0 0 0 0 0 0 8 0]]


In [25]:
# Die Anzahl Nullen in dem oberen Zeilenblock (3 Zeilen)
np.count_nonzero(a[0:3,:]==0)

19

In [27]:
# Zahlen austauschen
# von[i] wird durch nach[i] ersetzt

von  = [4, 6, 3]
nach = [3, 4, 6]

for i in range(len(von)):
    a[a == von[i]] = nach[i]
print(a)


[[4 0 0 0 8 0 0 0 1]
 [0 9 0 6 0 0 4 0 0]
 [0 0 0 0 9 0 0 0 2]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 4 0 0 0 0 5]
 [6 2 7 5 0 0 0 0 8]
 [0 0 0 0 7 0 0 0 0]
 [0 0 4 8 0 6 9 7 0]
 [0 0 0 0 0 0 0 8 0]]


In [28]:
# Gegeben zwei Matrizen su1, su2 - Test ob an allen Stellen, 
# an denen su1 den Wert k1 hat, su2 den Wert k2 hat.
su1 = np.array([[4, 0, 0, 0, 0, 8, 5, 0, 3],
       [0, 5, 9, 0, 2, 0, 0, 0, 7],
       [0, 0, 0, 7, 0, 0, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 0, 0, 2],
       [0, 0, 2, 1, 3, 9, 4, 0, 0],
       [0, 6, 0, 0, 0, 0, 7, 0, 0],
       [0, 0, 6, 0, 0, 2, 0, 0, 5],
       [0, 9, 7, 0, 8, 0, 0, 0, 0],
       [0, 0, 0, 0, 7, 0, 2, 0, 1]])
su2 = np.array([[9, 0, 0, 0, 0, 3, 2, 0, 1],
       [0, 2, 6, 0, 8, 0, 0, 0, 7],
       [0, 0, 0, 7, 0, 0, 0, 0, 0],
       [0, 0, 9, 0, 0, 0, 0, 0, 8],
       [0, 0, 8, 4, 1, 6, 9, 0, 0],
       [0, 5, 0, 0, 0, 0, 7, 0, 0],
       [0, 0, 5, 0, 0, 8, 0, 0, 2],
       [0, 6, 7, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 7, 0, 8, 0, 4]])

k1 = 5
k2 = 2
np.array_equal(su1==k1, su2==k2)

True

In [9]:
# Test, ob in Zeile mit Index 1 die Zahlen 9, 4 und 6 vorkommen
numbers = list(range(1,10))
np.isin([9,4,6], a[1,:]).all()

True

In [31]:
import numpy as np
def ulam(n):
    if n == 1:
        return np.array([1]).reshape(1,1)
    u = ulam(n-2)
    k = u[-1,-1]
    n1 = n-2
    rechts = np.arange(k+1,k+1+n)
 
    print(rechts)

In [32]:
print(ulam(3))

[2 3 4]
None


In [14]:
a = np.array(range(81)).reshape(9,9)

In [16]:
a[-1,-1]


80

In [18]:
np.arange(4,6)

array([4, 5])